# KDD 2020 
# Where do I walk?
Using RAPIDS to determine the shortest walking distance

## Load the modules

In [3]:
import cudf
from cuspatial import haversine_distance
from collections import OrderedDict
import numpy as np
import datetime as dt
import cugraph

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 4.63 ms


## Read in the data

In [4]:
dtypes = OrderedDict([
    ('OccupancyDateTime', 'date'),
    ('PaidOccupancy', 'int64'),
    ('BlockfaceName', 'str'),
    ('SideOfStreet', 'str'),
    ('SourceElementKey', 'int64'),
    ('ParkingTimeLimitCategory', 'int64'),
    ('ParkingSpaceCount', 'int64'),
    ('PaidParkingArea', 'str'),
    ('PaidParkingSubArea', 'str'),
    ('PaidParkingRate', 'int8'),
    ('ParkingCategory', 'str'),
    ('Location', 'str'),
    ('dow', 'int8')
])

df = cudf.read_csv(
    '../data/parking_MayJun2019.csv'
    , skiprows=1
    , dtype=list(dtypes.values())
    , names=list(dtypes.keys())
)

df = df[['SourceElementKey', 'Location']]

time: 3.36 s


In [5]:
def extractLon(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[0]
    return lon

def extractLat(location):
    lon = location.str.extract('([0-9\.\-]+) ([0-9\.]+)')[1]
    return lon
    

time: 634 µs


In [6]:
df['longitude'] = extractLon(df['Location']).astype('float')
df['latitude'] = extractLat(df['Location']).astype('float')

time: 3.81 s


In [7]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="todrabas_test")
location = geolocator.geocode("400 Broad St, Seattle, WA 98109") # SPACE NEEDLE

df['LON_Ref'] = location.longitude
df['LAT_Ref'] = location.latitude
df = df.dropna()

time: 1.12 s


In [8]:
distances = haversine_distance(
      df['longitude']
    , df['latitude']
    , df['LAT_Ref']
    , df['LON_Ref']
)

time: 29.6 ms


## As crow flies vs as people walk

### Get the parking locations

In [9]:
locations = df[['SourceElementKey', 'longitude', 'latitude']].drop_duplicates()
locations.head()

,SourceElementKey,longitude,latitude
4080,1001,-122.334694,47.602873
1336,1002,-122.334513,47.602949
4455,1006,-122.335143,47.603674
1026,1009,-122.336658,47.605018
3208,1010,-122.336447,47.605101


time: 506 ms


In [10]:
del df ## free up the GPU memory

time: 18.2 ms


### Read in the graph data
Thanks to John Murray for analyzing the map of King County roads and producing the data we will now use.

#### Download and unpack the data

In [11]:
import os

directory  = os.path.exists('../data')
archive    = os.path.exists('../data/king_county_road_graph_20190909.tar.gz')
file_graph = os.path.exists('../data/king_county_road_graph_20190909.csv')
file_nodes = os.path.exists('../data/king_county_road_nodes_20190909.csv')

if not directory:
    os.mkdir('../data')

if not archive:
    import wget, shutil
    
    wget.download('http://tomdrabas.com/data/seattle_parking/king_county_road_graph_20190909.tar.gz')
    shutil.move('king_county_road_graph_20190909.tar.gz', '../data/king_county_road_graph_20190909.tar.gz')
    
if not file_graph or not file_nodes:
    import tarfile

    tf = tarfile.open('../data/king_county_road_graph_20190909.tar.gz')
    tf.extractall(path='../data/')

time: 2.4 ms


In [12]:
!head ../data/king_county_road_graph_20190909.csv

node1,node2,LENGTH
89108,27652,5.02825
27652,89108,5.02825
27652,122930,112.417
122930,27652,112.417
36778,36779,48.2475
36779,36778,48.2475
26559,26559,48.3425
26559,26559,48.3425
2634,78382,372.325
time: 583 ms


In [13]:
!head ../data/king_county_road_nodes_20190909.csv

NodeID,Lon,Lat
1,-121.431505,47.331052
2,-121.430642,47.334726
3,-121.404812,47.288757
4,-121.408953,47.289157
5,-121.432249,47.292963
6,-121.432702,47.294297
7,-121.432724,47.329542
8,-121.367462,47.290593
9,-121.360285,47.29045
time: 577 ms


#### Let's read the road data

In [14]:
road_graph_data = cudf.read_csv('../data/king_county_road_graph_20190909.csv')
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')
road_graph_data['LENGTH'] = road_graph_data['LENGTH'] * 3 # convert to feet as the LENGHT was given in yards

time: 265 ms


In [15]:
road_nodes = cudf.read_csv('../data/king_county_road_nodes_20190909.csv')
road_nodes['NodeID'] = road_nodes['NodeID'].astype('int32')

time: 4.73 ms


Store the maximum of the `NodeID` so we can later append the parking nodes.

In [16]:
nodeId = road_nodes['NodeID'].max()
parking_nodes_idx = road_nodes['NodeID'].max() ## retain it so we can later filter the results to only parking locations
nodeId

127380

time: 2.4 ms


Move all the parking locations to host (via `.to_pandas()` method) so we can loop through all the ~1500 parking locations. Here, we also create an empty DataFrame that will hold the parking location nodes.

In [17]:
parking_locations = locations.to_pandas().to_dict('records')
parking_locations_nodes = cudf.DataFrame(columns=['NodeID', 'Lon', 'Lat', 'SourceElementKey'])

time: 12.8 ms


Let's process the parking data. In this naive approach we simply search for 3 closest road intersections to the parking location. We will then add two edges to the overall graph that link the parking node to the nearest intersection. This approach currently is producing some artifacts but we will deal with this in the future; in this example we want to simply explore the efficacy of finding the shortest path between the nodes in the graph.

In [18]:
for loc in parking_locations:
    nodeId = nodeId + 1

    ### calculate the distances to the road intersections
    road_nodes['Lon_REF'] = loc['longitude']
    road_nodes['Lat_REF'] = loc['latitude']
    road_nodes['Distance'] = haversine_distance(
          road_nodes['Lon']
        , road_nodes['Lat']
        , road_nodes['Lon_REF']
        , road_nodes['Lat_REF'])
    road_nodes['Distance'] = road_nodes['Distance']  * 0.621371 * 5280 # distance in feet as cuspatial returns kilometers

    nearest = (
        road_nodes
        .nsmallest(3, 'Distance') # connect to the nearest 3 intersections
    )
    
    nearest['node2'] = nodeId
    nearest = (
        nearest[['NodeID', 'node2', 'Distance']]
        .rename(columns={'NodeID': 'node1', 'Distance': 'LENGTH'})
    )
    road_graph_data = cudf.concat([road_graph_data, nearest]) ## append to the road_graph_data DataFrame

    ### add the parking node with its own unique NodeID
    rec = {
        'NodeID': nodeId
        , 'Lon': loc['longitude']
        , 'Lat': loc['latitude']
        , 'SourceElementKey': loc['SourceElementKey']
    }
    parking_locations_tmp = cudf.DataFrame(rec)
    parking_locations_nodes = cudf.concat([parking_locations_nodes, parking_locations_tmp])

time: 30.3 s


Now we can find the nearest intersections from the Space Needle!

In [19]:
road_nodes = (
    cudf
    .concat([road_nodes[['NodeID', 'Lon', 'Lat']], parking_locations_nodes])
    .reset_index(drop=True)
)

road_nodes['Lon_REF'] = location.longitude
road_nodes['Lat_REF'] = location.latitude

road_nodes['Distance'] = haversine_distance(
          road_nodes['Lon']
        , road_nodes['Lat']
        , road_nodes['Lon_REF']
        , road_nodes['Lat_REF'])

road_nodes['Distance'] = road_nodes['Distance']  * 0.621371 * 5280

space_needle_to_nearest_intersection = road_nodes.nsmallest(5, 'Distance') ### Space Needle is surrounded by around 5 road intersections hence we add 5
space_needle_to_nearest_intersection_dist = space_needle_to_nearest_intersection['Distance'].to_array()[0]

space_needle_to_nearest_intersection['node1'] = nodeId + 1
space_needle_to_nearest_intersection = (
    space_needle_to_nearest_intersection
    .rename(columns={'NodeID': 'node2', 'Distance': 'LENGTH'})
    [['node1', 'node2', 'LENGTH']]
)
road_graph_data = cudf.concat([space_needle_to_nearest_intersection, road_graph_data])
space_needle_to_nearest_intersection ### SHOW THE EDGES

,node1,node2,LENGTH
47756,128854,47757,175.906391
80448,128854,80449,200.062128
96739,128854,96740,261.056715
108797,128854,108798,277.221141
47827,128854,47828,301.715490


time: 37.1 ms


### The road graph

In [20]:
road_graph_data = road_graph_data.reset_index(drop=True)
road_graph_data['node1'] = road_graph_data['node1'].astype('int32')
road_graph_data['node2'] = road_graph_data['node2'].astype('int32')

g = cugraph.Graph()
g.from_cudf_edgelist(
    road_graph_data
    , source='node1'
    , destination='node2'
    , edge_attr='LENGTH'
)

time: 60.3 ms


Now we can use the `.sssp(...)` method from `cugraph` to find the shortest distances to parking spots from the Space Needle!

In [21]:
all_distances = cugraph.sssp(g, nodeId + 1)
distances = all_distances.query('vertex > @parking_nodes_idx and vertex < @nodeId + 1 and distance < 1000')

distances

,distance,vertex,predecessor
77439,909.096479,128160,47830
81273,755.699182,128506,47756
86558,959.897433,128223,47830
86559,957.417489,128224,47830
98400,471.997029,127585,47828
98401,451.645186,127586,47828
101163,991.776494,127884,47831
102503,771.775981,127912,47829
102504,770.279039,127913,47829
107655,955.897882,127666,47799


time: 329 ms


`cugraph` returns a DataFrame with vertex, distance to that vertex, and the total distance traveled to that vertex from the `nodeId + 1` node -- the Space Needle. Here, we unfold the full path.

In [22]:
# unfold -- create the whole path
closest_node = nodeId + 1
parking_cnt = distances['vertex'].count()

for i in range(parking_cnt):
    print('Processing record: {0}'.format(i))
    parking_node = distances.iloc[i].to_pandas()
    vertex = int(parking_node[1])
    predecessor = int(parking_node[2])
    
    if i == 0:
        paths = all_distances.query('vertex == @vertex')
    else:
        paths = cudf.concat([all_distances.query('vertex == @vertex'), paths])

    while vertex != closest_node:
        temp = all_distances.query('vertex == @predecessor')
        paths = cudf.concat([temp, paths])
        predecessor = temp['predecessor'].to_array()[0]
        vertex = temp['vertex'].to_array()[0]

Processing record: 0
Processing record: 1
Processing record: 2
Processing record: 3
Processing record: 4
Processing record: 5
Processing record: 6
Processing record: 7
Processing record: 8
Processing record: 9
Processing record: 10
Processing record: 11
Processing record: 12
Processing record: 13
Processing record: 14
time: 632 ms


### Charting the paths

In [23]:
paths['vertex'] = paths['vertex'].astype('int64')
paths['predecessor'] = paths['predecessor'].astype('int64')
paths = paths.drop_duplicates()

### process the data so we get the Lat/Lon back for both src and dest
### then move to host
paths_host = (
    paths
    .rename(columns={'vertex': 'NodeID'})
    .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
    .rename(columns={'NodeID': 'vertex', 'predecessor': 'NodeID'})
    .merge(road_nodes[['NodeID', 'Lat', 'Lon']], on='NodeID', how='left')
    .fillna({'Lat_y': location.latitude, 'Lon_y': location.longitude})
    [['vertex', 'Lat_x', 'Lon_x', 'Lat_y', 'Lon_y']]
    .query('vertex < @nodeId + 1')
    .to_pandas()
)

time: 127 ms


Get the information about the parking spots so we can create info boxes.

In [24]:
distances['vertex'] = distances['vertex'].astype('int64')
distances_host = (
    distances
    .rename(columns={'vertex': 'NodeID'})
    .merge(road_nodes[['NodeID', 'Lat', 'Lon', 'SourceElementKey']], on='NodeID')
    [['SourceElementKey', 'Lat', 'Lon', 'distance']]
    .to_pandas()
#     .to_dict('records')
)

time: 7.49 ms


In [25]:
info_box_template = """
<dl>
<dt>SourceElementKey</dt><dd>{SourceElementKey}</dd>
<dt>Distance</dt><dd>{distance:.0f} ft.</dd>
</dl>
"""

parking_info = [info_box_template.format(**parking) for parking in distances_host.to_dict('records')]

time: 893 µs


And... plot!

In [26]:
import gmaps
from ipywidgets.embed import embed_minimal_html

####################################################
##                                                ##
## CHANGE THE API CREDS IN THE GoogleMapsAPI.cred ##
##                                                ##
####################################################
with open('config/GoogleMapsAPI.cred', 'r') as f:
    gmaps_creds = f.read()

gmaps.configure(api_key=gmaps_creds) # Your Google API key, go to https://console.developers.google.com
parking_layer = gmaps.symbol_layer(
    distances_host[['Lat', 'Lon']], fill_color="green", stroke_color="green", scale=3, info_box_content=parking_info
)

destinations_layer = gmaps.symbol_layer(
    [[location.latitude, location.longitude]]
    , info_box_content=['DESTINATION']
    , scale=5
    , fill_color="red"
    , stroke_color="red"
)

lines_layer = gmaps.drawing_layer(features=[
    gmaps.Line(
          start = (path['Lat_x'], path['Lon_x'])
        , end   = (path['Lat_y'], path['Lon_y'])
        , stroke_weight=2
        , stroke_color="red"
    )
    for path in paths_host.to_dict('records')]
)

fig = gmaps.figure(layout={'height': '500px'})
fig.add_layer(parking_layer)
fig.add_layer(destinations_layer)
fig.add_layer(lines_layer)
embed_minimal_html('maps_rendered/map_walk_interim.html', views=[fig])

time: 108 ms
